# $k$近邻法

* $k$近邻法是一种基本分类与回归方法
* 只讨论分类问题中的$k$近邻法
* 假设给定一个训练数据集，其中的实例类别已定。分类时，对新的实例，根据其$k$个最近邻的训练实例的类别，通过多数表决等方式进行预测

# $k$近邻法

* 不具有显式的学习过程
* 实际上利用训练数据集对特征向量空间进行划分，并作为其分类的“模型”

## $k$近邻算法

1. 根据给定的距离度量，在训练集$T$中找出与$x$最邻近的$k$个点，涵盖这上$k$个点的$x$的邻域记作$N_k(x)$
1. 在$N_k(x)$中根据分类决策规则（如多数表决）决定$x$的类别$y$：
$$
y=\mathop{\arg\,\max}_{c_j}\sum_{x_i\in N_k(x)}I(y_i=c_j)
$$

课堂提问\_1：$k=1$的情形

## $k$近邻模型

* 模型由三个基本要素——距离度量、$k$值的选择和分类决策规则决定

### 距离度量

* 特征空间中两个实例点的距离是两个实例点相似程度的反映
* $x_i,x_j$的$L_p$距离定义为：
$$
L_p(x_i,x_j)=\biggl(\sum_{l=1}^n|x^{(l)}_i-x^{(l)}_j|^p\biggr)^{\frac{1}{p}}
$$

* 当$p=\infty$时，它是各个坐标距离的最大值，即
$$
L_\infty=\max_l|x^{(l)}_i-x^{(l)}_j|
$$

### 距离度量

1. 课堂提问\_2：图3.2中，$p=2$的形式；课堂提问\_3：图3.2中，$p=1$的形式
1. 例3.1：“所以$p$为任何值时，$L_p(x_1,x_2)=4$”。课堂提问\_4：为什么
1. “$p$大于等于3时，$x_3$是$x_1$的最近邻点”。课堂提问\_5：为什么

### $k$值的选择

* 如果$k = N$，那么无论输入实例是什么，都将简单地预测它属于在训练实例中最多的类
* 这时，模型过于简单，完全忽略训练实例中的大量有用信息，是不可取的
* 在应用中，$k$值一般取一个比较小的数值。通常釆用交叉验证法来选取最优的$k$值

## $k$近邻法的实现：$kd$树

* 实现$k$近邻法时，主要考虑的问题是如何对训练数据进行快速$k$近邻搜索
* 这点在特征空间的维数大及训练数据容量大时尤其必要

课堂提问\_6：linear scan

### 构造$kd$树

* $kd$树是二叉树，表示对$k$维空间的一个划分（partition）
* 构造$kd$树相当于不断地用垂直于坐标轴的超平面将$k$维空间切分，构成一系列的$k$维超矩形区域
* $kd$树的每个结点对应于一个$kd$维超矩形区域

In [1]:
from collections import namedtuple
from operator import itemgetter
from pprint import pformat

In [2]:
class Node(namedtuple("Node", "location left_child right_child")):
    def __repr__(self):
        return pformat(tuple(self))

In [3]:
def kdtree(point_list, depth: int = 0): 
    if not point_list:
        return None

    k = len(point_list[0])  # assumes all points have the same dimension
    # Select axis based on depth so that axis cycles through all valid values
    axis = depth % k 

    # Sort point list by axis and choose median as pivot element
    point_list.sort(key=itemgetter(axis))
    median = len(point_list) // 2

    # Create node and construct subtrees
    return Node(
        location=point_list[median],
        left_child=kdtree(point_list[:median], depth + 1), 
        right_child=kdtree(point_list[median + 1:], depth + 1), 
    )

In [4]:
def main():
    """Example usage"""
    point_list = [(7, 2), (5, 4), (9, 6), (4, 7), (8, 1), (2, 3)]
    tree = kdtree(point_list)
    print(tree)

In [5]:
if __name__ == "__main__":
    main()

((7, 2),
 ((5, 4), ((2, 3), None, None), ((4, 7), None, None)),
 ((9, 6), ((8, 1), None, None), None))


### 构造$kd$树

<center><img src="Kdtree_2d.svg" width="35%"></center>

### 构造$kd$树

<center><img src="Tree_0001.svg" width="50%"></center>

### 搜索$kd$树

1. 在$kd$树中找出包含目标点$x$的叶结点
1. 以此叶结点为“当前最近点”
1. 递归地向上回退，在每个结点进行以下操作：
    1. 如果该结点保存的实例点比“当前最近点”距离目标点更近，则以该实例点为“当前最近点”
    1. 检査该子结点的父结点的另一子结点$y$对应的区域$R$是否有更近的点
        1. 如果不相交，区域$R$被**eliminated**
        1. 如果相交，返回第1步，作用在以$y$为root的$kd$树上


### 搜索$kd$树

* 区域$R$被**eliminated**
* Since the hyperplanes are all axis-aligned this is implemented as a simple comparison to see whether the distance between the splitting coordinate of the search point and current node is lesser than the distance (overall coordinates) from the search point to the current best.